This notebook will create 1 dataframe from all four modules
Shuffle and Randomly partition that one dataframe 

60%train:

20%validation:  This data set is used to compare the performances of the prediction algorithms that were created based 
on the training set. We choose the algorithm that has the best performance.

Models:
1. Random Forest
2. 
3.
4.

##############
20%test: Once we have chosen our preferred prediction algorithm but we don't know yet how it's going to perform on 
completely new real-world data. So, we apply our chosen prediction algorithm on our test set in order to see how 
it's going to perform so we can have an idea about our algorithm's performance on new data.
##############

### What is Cross Validation ?

In [7]:
def generateDF(featuresFile, profiledFile):
    #reads featurefiles and projects 3 columns (ppn, sizeGB, Ytime)
    #reads profiledFile (the table from profiling) and reads all 55 columns
    #generates one dataframe with 58 columns and n rows (=size of profiled data from csv)
    import pandas as pd
    from IPython.display import display

    header='ft1 ft2 ft3 ft4 ft5 ft6 ft7 ft8 ft9 ft10 ft11 ft12 ft13 ft14 ft15 ft16 ft17 \
    ft18 ft24 ft25 ft19 ft39 ft20 ft33 ft21 ft35 ft22 ft23 ft34 ft36 ft37 ft38 ft40 ft41 \
    ft42 ft43 ft44 ft45 ft46 ft48 ft47 ft49 ft51 ft50 ft52 ft53 ft54 ft55 ft26 ft27 ft28 ft29 ft30 ft31 ft32'
    
    def value(item):
        return item[item.find('=')+1:]
    
    print("Reading features from %s" % featuresFile)
    df_features = pd.read_table(featuresFile, header=None, delimiter=',',
                       converters={i:value for i in range(55)},
                       names=header.split())
    df_features = df_features.astype(float)
    #print(df_features.head(2))
    
    ################################
    #read profiledFile
    df_profiled = pd.read_csv(profiledFile)
    if 'Unnamed: 0' in df_profiled.columns:
        del df_profiled['Unnamed: 0']
    rows = df_profiled.shape[0]
    
    #project columns of use
    df_profiled = df_profiled[['ppn','sizeGB','Y_time']]
    print("Reading profiled file %s\t: %s" % (profiledFile,str(df_profiled.shape)))
    
    ################################
    # create dataframe from featuresFile static program features with same #ROWs
    frames = [df_features for i in range(rows)]
    program_ft = pd.concat(frames)
    program_ft.reset_index(inplace=True)
    del program_ft['index']
    print("Shape of program static features\t\t\t\t\t: %s " % str(program_ft.shape))
    
    ################################
    # concatenate static features (55) with profiled data (3)
    ft_plus_profiled = pd.concat([df_profiled, program_ft], axis=1, join_axes=[df_profiled.index])
    ft_plus_profiled['y_time']=ft_plus_profiled['Y_time'] # so that y_time is last column
    del ft_plus_profiled['Y_time']
    print("Returning concatenated data frame (ft + profiled data)\t\t\t: %s" % str(ft_plus_profiled.shape))
    #display(ft_plus_profiled.tail(5))    
    
    return ft_plus_profiled

In [8]:
# stack two dataframes on top of each other
def stackDFs(df1, df2):
    import copy
    import pandas as pd
    
    frames = [copy.deepcopy(df1), copy.deepcopy(df2)]
    both = pd.concat(frames)
    both.reset_index(inplace=True)
    del both['index']
    return both

In [28]:
#
############################################################################################
############################################################################################
################################## cdhitdup ################################################
############################################################################################
############################################################################################
#

In [9]:
df_cdhitdup = generateDF('./features/cdhitdup_singlecmd_features.txt','./profiledcsvfiles/cdhitdup_ppn_sizeGB_Ytime.csv')
df_cdhitdup.shape

Reading features from ./features/cdhitdup_singlecmd_features.txt
Reading profiled file ./profiledcsvfiles/cdhitdup_ppn_sizeGB_Ytime.csv	: (32, 3)
Shape of program static features					: (32, 55) 
Returning concatenated data frame (ft + profiled data)			: (32, 58)


(32, 58)

In [13]:
df_cdhitdup.sizeGB.unique()

array([ 1.2 ,  1.7 ,  2.3 ,  2.9 ,  3.4 ,  4.  ,  4.5 ,  0.54])

In [29]:
######################################## cdhitdup
#scale the size of files in cdduphit

d=1024.0
whatisthesize_cddup = {0.54: 800/d,  1.2: 1608/d,  1.7: 2.4,  2.3: 3.2,   2.9: 4.0,\
                       3.4: 4.8,   4.0: 5.6,   4.5: 6.4}

test = df_cdhitdup.sizeGB.unique()
#create new col
df_cdhitdup['NEWsizeGB'] = df_cdhitdup['sizeGB'].map(whatisthesize_cddup)
#del old one
del df_cdhitdup['sizeGB']
#rename newly created col
df_cdhitdup.rename(columns={'NEWsizeGB':'sizeGB'}, inplace=True)

In [31]:
from IPython.display import display
display(df_cdhitdup[['sizeGB']])

,sizeGB
0,1.570312
1,2.400000
2,3.200000
3,4.000000
4,4.800000
5,5.600000
6,6.400000
7,0.781250
8,1.570312
9,2.400000


In [27]:
#
############################################################################################
############################################################################################
##################################### frhit ################################################
############################################################################################
############################################################################################
#

In [8]:
df_frhit = generateDF('./features/frhit_singlecmd_features.txt','./profiledcsvfiles/frhit_profiledData.csv')

Reading features from ./features/frhit_singlecmd_features.txt


KeyError: "['sizeGB' 'Y_time'] not in index"

In [3]:
######################################## frhit 

#map seqlen to sizeGB
d = 1024.0

whatisthesize_frhit = {'5M': 799/(d*d), '10M': 1.6/d,'15M': 2.4/d,'20M': 3.2/d,'25M': 4.0/d,\
                       '30M': 4.8/d,'35M': 5.5/d,'40M': 6.3/d}
#df_frhit['sizeGB'] = df_frhit['seqlen'].map(whatisthesize)
#del df_frhit['seqlen']

whatisthesize_frhit['15M']

0.00234375

In [26]:
#
############################################################################################
############################################################################################
##################################### velvetH ##############################################
############################################################################################
############################################################################################
#

In [ ]:
df_hvelvetH = generateDF('./features/velveth_singlecmd_features.txt','./profiledcsvfiles/velveth_profiledData.csv')

In [5]:
######################################## velvetH

#map seqlen to sizeGB
d = 1024.0

whatisthesize_velvetH = {'5M': 800/d, '10M': 1606/d,'15M': 2.4,'20M': 3.2,'25M': 4.0,\
                       '30M': 4.8,'35M': 5.6,'40M': 6.4}

In [25]:
#
############################################################################################
############################################################################################
##################################### velvetG ##############################################
############################################################################################
############################################################################################
#

In [ ]:
df_gvelvetG = generateDF('./features/velvetg_singlecmd_features.txt','./profiledcsvfiles/velvetg_profiledData.csv')

In [ ]:
######################################## velvetG 

#map seqlen to sizeGB
